# IR24 Project Testing Notebook

#### Import Libraries

In [1]:
from llama_index.core import SummaryIndex
from llama_index.readers.web import SimpleWebPageReader
from IPython.display import Markdown, display
import IR24_data

## Testing Data
### Wikipedia
https://en.wikipedia.org/wiki/History_of_Singapore

https://en.wikipedia.org/wiki/Early_history_of_Singapore 

https://en.wikipedia.org/wiki/Kingdom_of_Singapura 

https://en.wikipedia.org/wiki/Founding_years_of_modern_Singapore 

https://en.wikipedia.org/wiki/Singapore_in_the_Straits_Settlements 

https://en.wikipedia.org/wiki/Fall_of_Singapore

https://en.wikipedia.org/wiki/Japanese_occupation_of_Singapore 

https://en.wikipedia.org/wiki/Operation_Tiderace 

https://en.wikipedia.org/wiki/Colony_of_Singapore 

https://en.wikipedia.org/wiki/Self-governance_of_Singapore 

https://en.wikipedia.org/wiki/Singapore_in_Malaysia 

https://en.wikipedia.org/wiki/History_of_the_Republic_of_Singapore 

## Create .md files

In [9]:
urls = ["https://en.wikipedia.org/wiki/History_of_Singapore","https://en.wikipedia.org/wiki/Early_history_of_Singapore","https://en.wikipedia.org/wiki/Kingdom_of_Singapura","https://en.wikipedia.org/wiki/Founding_years_of_modern_Singapore","https://en.wikipedia.org/wiki/Singapore_in_the_Straits_Settlements","https://en.wikipedia.org/wiki/Fall_of_Singapore","https://en.wikipedia.org/wiki/Japanese_occupation_of_Singapore","https://en.wikipedia.org/wiki/Operation_Tiderace","https://en.wikipedia.org/wiki/Colony_of_Singapore","https://en.wikipedia.org/wiki/Self-governance_of_Singapore","https://en.wikipedia.org/wiki/Singapore_in_Malaysia", "https://en.wikipedia.org/wiki/History_of_the_Republic_of_Singapore"] 

for url in urls:
    IR24_data.html_to_markdown(url, "wikidata/"+url[30:]+".md")

Markdown content saved to wikidata/History_of_Singapore.md
Markdown content saved to wikidata/Early_history_of_Singapore.md
Markdown content saved to wikidata/Kingdom_of_Singapura.md
Markdown content saved to wikidata/Founding_years_of_modern_Singapore.md
Markdown content saved to wikidata/Singapore_in_the_Straits_Settlements.md
Markdown content saved to wikidata/Fall_of_Singapore.md
Markdown content saved to wikidata/Japanese_occupation_of_Singapore.md
Markdown content saved to wikidata/Operation_Tiderace.md
Markdown content saved to wikidata/Colony_of_Singapore.md
Markdown content saved to wikidata/Self-governance_of_Singapore.md
Markdown content saved to wikidata/Singapore_in_Malaysia.md
Markdown content saved to wikidata/History_of_the_Republic_of_Singapore.md


In [3]:
# Adapted from: https://docs.llamaindex.ai/en/stable/examples/data_connectors/WebPageDemo/
documents = SimpleWebPageReader(html_to_text=True).load_data(
    ["https://en.wikipedia.org/wiki/History_of_Singapore"]
)
print(documents[0])

index = SummaryIndex.from_documents(documents)
print(index)

Doc ID: https://en.wikipedia.org/wiki/History_of_Singapore
Text: Jump to content  Main menu  Main menu  move to sidebar hide
Navigation    * [Main page](/wiki/Main_Page "Visit the main page
\[z\]")   * [Contents](/wiki/Wikipedia:Contents "Guides to browsing
Wikipedia")   * [Current events](/wiki/Portal:Current_events "Articles
related to current events")   * [Random article](/wiki/Special:Random
"Visit a ran...
